In [ ]:
!pip uninstall openai

In [ ]:
!pip install llmx==0.0.15a0
!pip install cohere
!pip install openai
!pip install tiktoken

In [ ]:
import pandas as pd
import os
import json
import time
from openai import OpenAI

In [ ]:
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY")
)

In [ ]:
def batch_gpt(message_prompt):
    responses = []
    response = client.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[{"role": "user", "content": message_prompt}]
    )
    print(response.choices[0].message.content)
    r = response.choices[0].message.content
    responses.append(r)

    return r, responses

In [ ]:
def get_mess():
  mess = """
You need to find the existing discourse relations between 2 elementary discourse units. Discourse relations between 2 discourse units can be explicit or implicit.
I will give you a text and 2 elementary discourse units extracted from that text and then i will provide you with all the 12 possible discourse relation types from which you should choose one. In the context of the example discourse units, discourse_unit_1 comes before discourse_unit_2. From the give relaions_types you should  determine the type of existing discourse relation between 2 given example elementary discourse units.
Return the result in json (example: {"discourse_relation" : "found_discourse_relation"} with no addition explanations.
      """

  return mess

In [ ]:
dataset = pd.read_csv('/local_path/eng.dep.covdtb_test.rels.csv', sep='\t')
df = pd.DataFrame(columns=['EDU_1', 'EDU_2', 'Real discourse relation', 'ChatGPT discourse relation'])
unprocessed_data = pd.DataFrame(columns=['unit1_sent', 'unit2_sent', 'doc_id', 'label'])
dataset = dataset[dataset['label'] != 'elaboration']

In [ ]:
dataset.head()

,doc,unit1_toks,unit2_toks,unit1_txt,unit2_txt,s1_toks,s2_toks,unit1_sent,unit2_sent,dir,orig_label,label
1,00c862a6402381a8541bcffe843696e059095300,1-18,23-32,"In this study , we investigated turkey reoviru...",RT - PCR for TReoV detected 3.3 % positive sam...,1-22,23-41,"In this study , we investigated turkey reoviru...",RT - PCR for TReoV detected 3.3 % positive sam...,1<2,findings,findings
2,00c862a6402381a8541bcffe843696e059095300,23-32,33-41,RT - PCR for TReoV detected 3.3 % positive sam...,and TReoV was successfully isolated in Vero ce...,23-41,23-41,RT - PCR for TReoV detected 3.3 % positive sam...,RT - PCR for TReoV detected 3.3 % positive sam...,1<2,joint,joint
5,00c862a6402381a8541bcffe843696e059095300,"77-78,83",84-102,Phylogenetic analysis <*> showed,that the Brazilian TReoV isolates clustered in...,77-108,77-108,Phylogenetic analysis based on S3 gene showed ...,Phylogenetic analysis based on S3 gene showed ...,1>2,attribution,attribution
6,00c862a6402381a8541bcffe843696e059095300,"77-78,83",79-82,Phylogenetic analysis <*> showed,based on S3 gene,77-108,77-108,Phylogenetic analysis based on S3 gene showed ...,Phylogenetic analysis based on S3 gene showed ...,1<2,manner-means,manner-means
7,00c862a6402381a8541bcffe843696e059095300,42-58,84-102,Histological analysis of positive bursa of Fab...,that the Brazilian TReoV isolates clustered in...,42-58,77-108,Histological analysis of positive bursa of Fab...,Phylogenetic analysis based on S3 gene showed ...,1<2,joint,joint


In [ ]:
dataset.shape

(1289, 12)

In [ ]:
data_with_texts = pd.read_csv('/local_path/eng_dep_covdtb_texts_test.csv')
data_with_texts.head()

,Document ID,Document Abstract
0,00c862a6402381a8541bcffe843696e059095300,"In this study , we investigated turkey reoviru..."
1,0146a0b36f80ebcb868c7e16fb60c1a9d2afbf83,This is an extensive review on epiphytic plant...
2,0232f18e7bbf3192e9e34da1c0abe2a20b2ba1e4,"Herein , the LASSBio Chemical Library is prese..."
3,09bdbea1700a3826acc33484ed51e2fa8c23f047,Data - centric models of COVID-19 have been tr...
4,0b362bc7e9278b56c2740ab7bc10e91c036f835a,We study seasonal epidemic spreading in a susc...


In [ ]:
message_part1 = get_mess()
message_part2 = {
      'findings',
      'joint',
      'attribution',
      'manner-means',
      'enablement',
      'cause-result',
      'background',
      'comparison',
      'textual-organization',
      'temporal',
      'condition'
  }

for i in range(0, len(dataset)):
  print()
  doc_id = dataset.iloc[i]['doc']
  sentence_1 = dataset.iloc[i]['unit1_txt'] 
  sentence_2 = dataset.iloc[i]['unit2_txt']  
  real_label = dataset.iloc[i]['label']

  row = data_with_texts[data_with_texts['Document ID'] == doc_id]
  document_text = row['Document Abstract'].values[0]

  message_part3 = {
      "text" : document_text
  }

  message_part4 = {
        "discourse_unit_1" : sentence_1,
        "discourse_unit_2" : sentence_2
  }

  prompt_message = message_part1 + '\n' + 'relation_types = ' + str(message_part2) + '\n' + '\nexample_text = ' + str(message_part3) + '\n' + '\nexample_discourse_units = ' + str(message_part4)
  print(str(i) + " " + str(message_part4))
  try:
    result, response = batch_gpt(prompt_message)
    result = result.replace('json', '')
    result = result.replace('`', '')
    result = result.strip()
    json_result = json.loads(result)
    gpt_relation = json_result['discourse_relation']
  except Exception as e:
    print("ERROR")
    print(e)
    unprocessed_data.loc[len(unprocessed_data)] = {'unit1_txt': sentence_1, 'unit2_txt': sentence_2, 'doc_id': doc_id, 'label': real_label}
  else:
    new_row = {'EDU_1': sentence_1, 'EDU_2': sentence_2, 'Real discourse relation': real_label, 'ChatGPT discourse relation': gpt_relation}
    df.loc[len(df)] = new_row
    if i % 300 == 0:
      time.sleep(10)

In [ ]:
df.tail()

,EDU_1,EDU_2,Real discourse relation,ChatGPT discourse relation
1284,The dissection room experience provides an opp...,Hence in every stage of dissection room activi...,cause-result,cause-result
1285,Hence in every stage of dissection room activi...,so as to set a standard for the students,enablement,cause-result
1286,The present review is an attempt,to collate the recommendations,enablement,manner-means
1287,which needs to be adhered to,while receiving the human body of a donor and ...,temporal,condition
1288,The present review is an attempt,The intended purpose of this article is to sup...,enablement,joint


In [ ]:
unprocessed_data.to_csv('gpt4_eng_dep_covdtb_test_unprocessed_prompt_messagges_elaboration_excluded.csv', index=False)
df.to_csv('gpt4_eng_dep_covdtb_test_results_elaboration_excluded.csv', index=False)

In [ ]:
df['ChatGPT discourse relation'].unique()

array(['background', 'joint', 'attribution', 'manner-means',
       'cause-result', 'purpose', 'enablement', 'contrast', 'comparison',
       'textual-organization', 'findings', 'condition', 'temporal',
       'concession', 'restatement', 'generalization'], dtype=object)

In [ ]:
df['Real discourse relation'].unique()

array(['findings', 'joint', 'attribution', 'manner-means', 'enablement',
       'cause-result', 'background', 'comparison', 'textual-organization',
       'temporal', 'condition'], dtype=object)

In [ ]:
not_included = ['contrast', 'purpose', 'concession', 'restatement', 'generalization']
br = 0
for i in range(len(df)):
  if df.iloc[i, 3] in not_included:
    br += 1
print(br)

102
